In [ ]:
## creating heatmaps of specific diseases

In [3]:
import seaborn as sns
import numpy as np
import pandas as pd
from pprint import pprint
from sqlalchemy import create_engine
import datashader as ds
import matplotlib.pyplot as plt
from colorcet import fire, blues
from datashader.utils import export_image
from datashader import transfer_functions as tf

In [4]:
POSTGRES_USERNAME = "sahil"
POSTGRES_PASSWORD = "zxcvbnm"
POSTGRES_DBNAME = "postgres"
POSTGRES_DB_ADDRESS = "localhost"

url = 'postgresql://{}:{}@{}:{}/{}'.format(POSTGRES_USERNAME, POSTGRES_PASSWORD, POSTGRES_DB_ADDRESS, 5432, POSTGRES_DBNAME)
engine = create_engine(url)

In [5]:
def remove_whitespace(x):
    """
    Helper function to remove any blank space from a string
    x: a string
    """
    try:
        # Remove spaces inside of the string
        x = "".join(x.split())

    except:
        pass
    return x

In [6]:
def create_image(merged_df, 
                 x_coord='x', 
                 y_coord='y',
                 save_name = None):
    
    canvas = ds.Canvas(640, 640)
    agg = canvas.points(merged_df,'x','y')
    raster_image = tf.shade(agg, cmap=fire)
    
    # uncommend the line below to have more coarse apperance
    image = tf.set_background(tf.dynspread(raster_image), "black")
    
    
    if(save_name is not None):
        export_image(image, save_name)
        
    return image

In [7]:
#select pdpi of 2016-2017
pdpi_tables = pd.read_sql_query("select tablename from pg_tables where tablename ~* 'pdpi_(2016|2017).*' order by tablename asc;"
                  , con=engine)

pdpi_tables.head()

,tablename
0,pdpi_201601
1,pdpi_201602
2,pdpi_201603
3,pdpi_201604
4,pdpi_201605


In [9]:
addr_index_table = pd.read_sql_table(table_name="addr_index", con = engine)
addr_index_table.postcode = addr_index_table.postcode.apply(remove_whitespace)
print(addr_index_table.shape)
addr_index_table.head()

(13830, 7)


,index,practice,postcode,latitude,longitude,x,y
0,0,A81001,TS181HU,54.561625,-1.318986,-146828.849883,7.277247e+06
1,1,A81002,TS182AW,54.569163,-1.313928,-146265.846882,7.278694e+06
2,2,A81003,TS268DB,54.685239,-1.217599,-135542.500670,7.301016e+06
3,3,A81004,TS13BE,54.571809,-1.232274,-137176.153683,7.279203e+06
4,4,A81005,TS147DJ,54.532605,-1.055428,-117489.720853,7.271678e+06


In [10]:
import time

In [12]:
%%time

# select practice for  chapter 03 - respiratory dieseaes

for t in pdpi_tables.tablename:
    print(t)

    # query the database
    query = "select practice, period from " + str(t) + " where chapter  like '03'";  
    
    # time the query
    start_time = time.time()
    df = pd.read_sql_query(query, con=engine)
    end_time = time.time()
    print("Query took ", start_time-end_time)
    
    
    
    # find the locations of the practices
    start_time = time.time()
    merged_df = pd.merge(df, addr_index_table, on="practice", how="left")
    end_time = time.time()
    print("Merging took ", start_time-end_time)
    
    print("Percentage of missing ", (merged_df['x'].isnull().sum() * 100) / (df.shape[0]))
    
    # create heatamp
    create_image(merged_df, 'x', 'y', str(t))
    
#     del df
#     del merged_df
    
    


pdpi_201601
Query took  -15.403339385986328
  practice  period  index postcode   latitude  longitude              x  \
0   N81002  201601   6286   SK91PA  53.323034  -2.235179 -248819.023524   
1   N81002  201601   6286   SK91PA  53.323034  -2.235179 -248819.023524   

              y  
0  7.042976e+06  
1  7.042976e+06  
Merging took  -0.5476455688476562
Percentage of missing  1.7672266135478387
pdpi_201602
Query took  -17.42759871482849
  practice  period  index postcode   latitude  longitude              x  \
0   P81707  201602   6859   BB11LX  53.746498  -2.464808 -274381.132167   
1   P81707  201602   6859   BB11LX  53.746498  -2.464808 -274381.132167   

              y  
0  7.122291e+06  
1  7.122291e+06  
Merging took  -0.11299777030944824
Percentage of missing  1.7468366910444604
pdpi_201603
Query took  -14.959887027740479
  practice  period  index postcode  latitude  longitude   x   y
0   Y00327  201603   8630   CH21HJ       NaN        NaN NaN NaN
1   Y00327  201603   8630   

In [16]:
###
### Creating timelapse
###


import imageio
import glob
images = []
filenames = glob.glob("./pdpi_*.png") 
print(filenames)
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('heatmap_timelapse.gif', images)

['./pdpi_201602.png', './pdpi_201701.png', './pdpi_201603.png', './pdpi_201611.png', './pdpi_201608.png', './pdpi_201705.png', './pdpi_201703.png', './pdpi_201706.png', './pdpi_201607.png', './pdpi_201612.png', './pdpi_201702.png', './pdpi_201609.png', './pdpi_201605.png', './pdpi_201704.png', './pdpi_201606.png', './pdpi_201610.png', './pdpi_201601.png', './pdpi_201604.png']


In [17]:
from IPython.display import HTML
HTML('<img src="heatmap_timelapse.gif">')
